## PCA as a fault detection model
Multiway-PCA can be used for fault detection when combining it with two types of measurements for errors: The squared prediction error (SPE) and the Hotellings T^2. This notebooks shows how to use the model implemented in this package. Additionaly, gridsearch is performed to find an optimal set of hyperparameters.

In [1]:
from fermfaultdetect.data.utils import load_batchset, dataloader
from fermfaultdetect.utils import get_simulation_dir
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from fermfaultdetect.utils import get_models_dir
from sklearn.decomposition import PCA
from fermfaultdetect.fault_detect_models.ml_models import pca_fdm
from fermfaultdetect import model_evaluation as eval
from fermfaultdetect.visualizations import visualize
from fermfaultdetect.model_evaluation import plot_example_set
from datetime import datetime
import pickle
import json

### Load training and validation data

In [2]:
seed = 42 # set seeding

sim_dir = get_simulation_dir() # get directory of simulation data

###############################
model_name = "FILL_IN_MODEL_NAME" # set the name of model (e.g. date or specific name)
train_set_name = "FILL_IN_TRAINING_SET_NAME"
val_set_name = "FILL_IN_VALIDATION_SET_NAME"
###############################

train_path = os.path.join(sim_dir, train_set_name)
val_path = os.path.join(sim_dir, val_set_name)

# set directory to save model and metrics
model_dir = os.path.join(get_models_dir(), model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

train_set = load_batchset(train_path)
val_set = load_batchset(val_path)

# Load train data into dataloader and standardize
target_cols = ['defect_steambarrier', 'steam_in_feed', 'blocked_spargers', 'airflow_OOC', 'OUR_OOC', 'no_fault'] # set target columns
train_dl = dataloader(batchset = train_set[:], seed=seed)
train_dl.shuffle_batches()
train_dl.standardize_data(exclude_cols=target_cols)

# Load test data into dataloader and standardize
val_dl = dataloader(batchset = val_set[:], seed=seed)
#test_dl.shuffle_batches()
val_dl.import_standardization(train_dl)
val_dl.standardize_data(exclude_cols=target_cols)

# Retrieve data from dataloader with separate and fused target columns
train_X, _ = train_dl.get_data(split_batches=False, target_cols=target_cols, separate_target_matrix=True, fuse_target_cols=True)
val_X, val_Y = val_dl.get_data(split_batches=False, target_cols=target_cols, separate_target_matrix=True, fuse_target_cols=True)
_, val_Y_unfused = val_dl.get_data(split_batches=False, target_cols=target_cols, separate_target_matrix=True, fuse_target_cols=False)

### Determine number of PCs
As a rule of thumb, one should choose a number of PC that explain at least 95 % of the variance.

In [ ]:
# Training PCA model
pca = PCA()
pca.fit(train_X)

# Calculating cumulative explained variance
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Plotting the cumulative explained variance to determine the number of components to retain
colors = visualize.get_thesis_colors()
plt.figure(figsize=(6, 4))
plt.plot(range(0, len(cumulative_explained_variance)), cumulative_explained_variance, marker='o', color=colors["green"])
plt.xlabel('Number of principal components [-]')
plt.ylabel('Cumulative explained variance [-]')
#plt.title('PCA Cumulative Explained Variance')
#plt.grid(True)
plt.axhline(y=0.95, color=colors["red"], linestyle='--')  # 95% explained variance line
#plt.axhline(y=0.90, color='g', linestyle='--')  # 90% explained variance line

# Save the plot
plt.savefig(os.path.join(model_dir, "PCA_Cumulative_Explained_Variance.png"), dpi=300)
plt.show()


cumulative_explained_variance

## Optimize detection threshold alpha and moving time window through gridsearch

### Run gridsearch

In [4]:
###
pc = 3 # choose based on previous section
###

# Define the range of values for hyperparameters
a_values = np.around(np.linspace(0.01, 0.4, 14), 2) #round to 2 decimals
mw_values = np.linspace(1, 30, 6, dtype=int)


# Set up results list
results = []

# Loop through all possible combinations of 'a' and 'mw'
for a in a_values:
    for mw in mw_values:
        #accuracy = pca_model_accuracy(train_X, test_X, test_Y, pc=pc, alpha=a, mw=mw)
        pca_model = pca_fdm(pc=pc, alpha=a, mw=mw)
        pca_model.calibrate(train_X)
        accuracy = pca_model.prediction_accuracy(val_X, val_Y)
        results.append({
            'a': a,
            'mw': mw,
            'accuracy': accuracy
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Pivot the DataFrame for heatmap plotting
pivot_table = results_df.pivot(index='a', columns='mw', values='accuracy')

# Save the pivot table to a CSV file
heatmap_path = os.path.join(model_dir, "pca_gridsearch_heatmap_"+model_name+".csv") # can be passed to metrics_table
pivot_table.to_csv(heatmap_path)

### Visualize gridsearch

In [ ]:
# Print optimal hyperparameters
best_row = results_df.loc[results_df['accuracy'].idxmax()]
print(f"Optimal parameters: accuracy = {best_row['accuracy']:.3f}, a = {best_row['a']:.3f}, mw = {best_row['mw']}")

# Plotting the results using seaborn heatmap
plt.figure(figsize=(10, 8))
ax = sns.heatmap(pivot_table, annot=False, cmap=visualize.get_hotcold_colormap(), fmt=".3f")
#plt.title('PCA Model Accuracy Heatmap')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
plt.xlabel(r'Moving time window $\it{n}$ [-]')
plt.ylabel('Threshold α [-]')
# Save the plot
plt.savefig(os.path.join(model_dir, "PCA_Model_Accuracy_Heatmap.png"))
plt.show()

### Analyse and save optimal MPCA model

In [ ]:
### Select optimal threshold and moving window ###
alpha = 0.5
mw = 1

pca_model_best = pca_fdm(pc=3, alpha=best_row['a'], mw=best_row['mw'].astype(int))
pca_model_best.calibrate(train_X)
predictions_best = pca_model_best.predict(val_X)

filepath = os.path.join(model_dir, "pca_test_opt.csv")
metrics = eval.metrics_table_oneclass(val_Y_unfused, predictions_best["fault"], save_path=filepath)
eval.visualize_metrics(metrics)

### Save model configuration

In [24]:
# Save PLS model
filename = 'model.pkl' # set model name
save_path = os.path.join(model_dir, filename)
with open(save_path, 'wb') as file:
    pickle.dump(pca_model_best, file)

# Create and save config
config_model = {
    "model": "MPCA",
    "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "train_set": train_set_name,
    "name": model_name,
    "n_components": pc,
    "alpha": alpha,
    "moving time window": mw
}

# Save the model config as a json file
config_name = "config.json"
config_path = os.path.join(model_dir, config_name)
with open(config_path, 'w') as json_file:
    json.dump(config_model, json_file, indent=4)

### Show performance with exemplatory batch data

In [ ]:
example_setname = "FILL_IN_EXAMPLE_SET_NAME"
visualize.set_plot_params(high_res=True)
plot_example_set(model=pca_model_best, train_dl=train_dl, setname=example_setname, parameter_plotted="weight", combined_figure=False)